In [78]:
!pip install pdfplumber
!pip install pymupdf

# Upload Sidra File/ Files here

In [85]:
from google.colab import files
import os

uploaded = files.upload()

Saving J_18_S1 L_AM_Site_2040 - New_Right_drive.pdf to J_18_S1 L_AM_Site_2040 - New_Right_drive (4).pdf
Saving J_18_S2 L_AM_Site_2040 - New_Right_drive.pdf to J_18_S2 L_AM_Site_2040 - New_Right_drive (4).pdf
Saving Jn-23_Site_2040 - New_Right_drive.pdf to Jn-23_Site_2040 - New_Right_drive (4).pdf
Saving Jn-24_Site_2040 - New_Right_drive.pdf to Jn-24_Site_2040 - New_Right_drive (4).pdf
Saving Jn-25_Site_2040 - New_Right_drive.pdf to Jn-25_Site_2040 - New_Right_drive (4).pdf


# Run

In [86]:
def categorize_value(B17):
    if B17 <= 10:
        return "A"
    elif 10 < B17 <= 15:
        return "B"
    elif 15 < B17 <= 25:
        return "C"
    elif 25 < B17 <= 35:
        return "D"
    elif 35 < B17 <= 50:
        return "E"
    else:
        return "F"

In [87]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re
import fitz
import shutil
import warnings
import logging

In [88]:
output_folder = "output_pages"
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

count = 0
os.makedirs(output_folder, exist_ok=True)
for filename in uploaded.keys():
    doc = fitz.open(filename)
    for i, page in enumerate(doc):
        text = page.get_text("text")
        lines = text.splitlines()
        if lines and "MOVEMENT SUMMARY" in lines[0]:
            new_doc = fitz.open()
            new_doc.insert_pdf(doc, from_page=i, to_page=i)
            output_path = os.path.join(output_folder, f"Sidra_JN_{count+1}.pdf")
            new_doc.save(output_path)
            new_doc.close()
            count += 1

In [89]:
Output = pd.DataFrame(columns= ['Scenario','Location', 'V/C', 'Delay', 'LOS', 'Max_Queue_Length', 'Approach', 'W_Movement', 'W_Delay', 'W_V/C','W_LOS'] )
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    all_text = []
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                all_text.append(f"Page {page_num}:\n{page_text}\n")
    full_text = "\n".join(all_text)
    data = []
    for line in full_text.split('\n'):
        data.append(line)
    pattern = r"Site: (\S+)\s\[(.*?)\s\(Site Folder: (.*?)\)\]"
    match = re.search(pattern, data[2])
    if match:
        site_number = match.group(1)
        tmc_number = match.group(2)
        scenario = match.group(3)
    start_index = None
    for i, line in enumerate(data):
        if 'All Vehicles' in line:
            start_index = i
            break
    filtered_data = data[:start_index+1]
    split_data = [row.split() for row in filtered_data[12:]]
    df= pd.DataFrame(split_data)
    Delay = df.iloc[-1,7]
    w_vc = df.iloc[-1,6]
    LOS = categorize_value(float(Delay))
    def assign_direction(df):
        direction = None
        directions = []
        for i, row in df.iterrows():
            if isinstance(row[0], str):
                temp_direction = ""

                if 'South' in row[0]:
                    temp_direction += "South"
                if 'East' in row[0]:
                    temp_direction += "East"
                if 'North' in row[0]:
                    temp_direction += "North"
                if 'West' in row[0]:
                    temp_direction += "West"


                if 'Approach' in row[0]:
                    temp_direction = direction


                if temp_direction:
                    direction = temp_direction

            directions.append(direction)

        df['Approach Direction'] = directions
        return df

    df = assign_direction(df)
    Que = df[df[0].apply(lambda x: len(str(x)) <= 3)].iloc[:-1,:]
    Que = Que.reset_index(drop=True)
    Que.iloc[:, 13] = pd.to_numeric(Que.iloc[:, 13], errors='coerce')
    index_of_max = Que.iloc[:, 13].idxmax()
    Approch = [Que.iloc[index_of_max, -1] if Que.iloc[index_of_max, 13] > 0 else 'No Queue'][0]
    Max_Queue_Length = Que.iloc[index_of_max, 13]
    Que.iloc[:, [4, 8, 9]] = Que.iloc[:, [4, 8, 9]].apply(pd.to_numeric, errors='coerce')

    W_delay = Que.iloc[:, 9].max()
    W_los = categorize_value(float(W_delay))

    max_i_delay = Que[Que.iloc[:, 9] == W_delay].index.tolist()

    if len(max_i_delay) > 1:
        m_vc = Que.loc[max_i_delay, Que.columns[8]].max()
        max_i_vc = Que.loc[max_i_delay][Que.iloc[:, 8] == m_vc].index.tolist()

        if len(max_i_vc) > 1:
            m_i_vol = Que.loc[max_i_vc, Que.columns[4]].idxmax()
            W_vc = Que.loc[m_i_vol, Que.columns[8]]
            W_Movement = Que.loc[m_i_vol, Que.columns[-1]] + ' ' + Que.loc[m_i_vol, Que.columns[1]]
        else:
            idx = max_i_vc[0]
            W_vc = Que.loc[idx, Que.columns[8]]
            W_Movement = Que.loc[idx, Que.columns[-1]] + ' ' + Que.loc[idx, Que.columns[1]]
    else:
        idx = max_i_delay[0]
        W_vc = Que.loc[idx, Que.columns[8]]
        W_Movement = Que.loc[idx, Que.columns[-1]] + ' ' + Que.loc[idx, Que.columns[1]]
    new_row = {
        'Scenario': scenario,
        'Location': tmc_number,
        'V/C': w_vc,
        'Delay': Delay,
        'LOS': LOS,
        'Max_Queue_Length': Max_Queue_Length,
        'Approach': Approch,
        'W_Movement':W_Movement,
        'W_Delay': W_delay,
        'W_V/C': W_vc,
        'W_LOS': W_los
    }

    Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)
    warnings.filterwarnings("ignore")
    logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Report

In [90]:
Output

,Scenario,Location,V/C,Delay,LOS,Max_Queue_Length,Approach,W_Movement,W_Delay,W_V/C,W_LOS
0,AM,Jn-24,0.256,3.5,A,8.7,South,South R2,3.8,0.256,A
1,AM,Jn-25,0.336,4.2,A,13.0,East,North L2,5.3,0.197,A
2,AM,Jn-23,0.608,8.9,A,36.4,South,East L2,10.7,0.556,B
3,Diamond Interchange,J_18_S2 L_AM,0.009,1.2,A,0.2,West,West L2,3.3,0.007,A
4,Diamond Interchange,J_18_S1 L_AM,0.009,1.2,A,0.2,East,East L2,3.3,0.007,A


In [ ]:
Output.to_excel('Sidra_Summary_Report.xlsx', index=False)